# 장애인콜택시 대기시간 예측
## 단계1.데이터 전처리

## 0.미션

* 1.데이터를 탐색하며 정보 획득
    * 데이터는 2015-01-01 ~ 2022-12-31 까지의 서울 장애인 콜택시 운행 정보입니다. 우리는 평균대기시간을 예측하고자 합니다. 
    * 요일, 월, 계절, 연도별 운행 정보에 어떤 주기와 변화가 있는지 탐색해 봅시다.
* 2.분석을 위한 데이터 구조 만들기
    * 문제정의 : 
        * 전 날 콜택시 운행이 종료되었을 때, 다음 날 대기시간을 예측하고자 합니다.

        * 만약 다음 날 대기시간을 예측할 수 있다면, 일정 범위 내에서 배차를 조절할 수 있을 뿐만 아니라, 향후 교통약자의 이동 편의 증진을 위한 정책 수립 및 개선에 기여할 수 있습니다. 
    * 이를 위한 데이터 구조를 만들어 봅시다.
        * 분석 단위는 일별 데이터 입니다.
        * 주어진 데이터 : 장애인 콜택시 운행 정보, 서울시 날씨
        * 날씨 데이터는 실제 측정값이지만, 다음 날에 대한 예보 데이터로 간주합니다. 
            * 예를 들어, 
                * 2020-12-23 의 날씨 데이터는 전 날(12월22일) 날씨예보 데이터로 간주하여 분석을 수행합니다.
                * 2020-12-22일의 장애인 이동 데이터로 23일의 대기시간을 예측해야 하며, 이때 고려할 날씨데이터는 23일 데이터 입니다.
        * 장애인 이동 데이터를 기준으로 날씨 데이터를 붙여서 만듭시다.
        * 휴무일 데이터는 패키지를 통해서 다운받아 사용합니다.
    * Feature Engineering
        * 대기시간에 영향을 주는 요인을 도출하고(가설수립) 이를 feature로 생성합시다.
        * 주어진 그대로의 데이터가 아닌 새로운 feature를 생성해 봅시다.
            * 날짜와 관련된 feature : 요일, 월, 계절 ... 
            * 시계열 특성이 반영된 feature : 최근 7일간의 평균 대기시간 ...




## 1.환경설정

* 세부 요구사항
    - 경로 설정 : 다음의 두가지 방법 중 하나를 선택하여 폴더를 준비하고 데이터를 로딩하시오.
        * 1) 로컬 수행(Ananconda)
            * 제공된 압축파일을 다운받아 압축을 풀고
            * anaconda의 root directory(보통 C:/Users/< ID > 에 project 폴더를 만들고, 복사해 넣습니다.
        * 2) 구글콜랩
            * 구글 드라이브 바로 밑에 project 폴더를 만들고, 
            * 데이터 파일을 복사해 넣습니다.
    - 라이브러리 설치 및 로딩
        * requirements.txt 파일로 부터 라이브러리 설치
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다. 
        * 필요하다고 판단되는 라이브러리를 추가하세요.

### (1) 경로 설정

#### 1) 로컬 수행(Anaconda)
* project 폴더에 필요한 파일들을 넣고, 본 파일을 열었다면, 별도 경로 지정이 필요하지 않습니다.

In [ ]:
# path = 'C:/Users/User/project/'

#### 2) 구글 콜랩 수행

* 구글 드라이브 연결

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# path = '/content/drive/MyDrive/project/'

### (2) 라이브러리 설치 및 불러오기

#### 1) 설치

* requirements.txt 파일을 아래 위치에 두고 다음 코드를 실행하시오.
    * 로컬 : 다음 코드셀 실행
    * 구글콜랩 : requirements.txt 파일을 왼쪽 [파일]탭에 복사해 넣고 다음 코드셀 실행

In [ ]:
# !pip install -r requirements.txt

#### 2) 라이브러리 로딩

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import joblib

# 더 필요한 라이브러리를 아래에 추가합니다.
import warnings
warnings.filterwarnings(action='ignore')


### (3) 데이터 불러오기
* 주어진 데이터셋
    * 장애인 콜택시 운행 정보 : open_data.csv
    * 날씨 데이터 : weather.csv

#### 1) 데이터로딩

In [3]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
taxi = pd.read_csv('open_data.csv')
weather = pd.read_csv('weather.csv')

In [4]:
taxi.describe().T

,count,mean,std,min,25%,50%,75%,max
차량운행,2922.0,418.701916,133.755162,132.0,278.0,454.0,522.00,1413.0
접수건,2922.0,3925.607803,1509.741713,527.0,2162.5,4720.5,5110.00,6182.0
탑승건,2922.0,3283.895277,1249.165876,462.0,1779.0,3932.5,4241.00,5189.0
평균대기시간,2922.0,40.305681,14.098550,17.2,29.6,38.2,48.60,96.1
평균요금,2922.0,2304.272416,107.260080,2131.0,2228.0,2257.0,2400.75,2733.0
평균승차거리,2922.0,9253.500000,1019.198152,7672.0,8521.0,8821.5,10153.00,14136.0


#### 2) 기본 정보 조회

In [3]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
# 전체 데이터의 행,열 개수 확인
print(taxi.shape)
print(weather.shape)

(2922, 7)
(4018, 7)


In [4]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
# 전체 데이터의 모든 변수 확인
display(taxi.info())
display(weather.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2922 entries, 0 to 2921
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기준일     2922 non-null   object 
 1   차량운행    2922 non-null   int64  
 2   접수건     2922 non-null   int64  
 3   탑승건     2922 non-null   int64  
 4   평균대기시간  2922 non-null   float64
 5   평균요금    2922 non-null   int64  
 6   평균승차거리  2922 non-null   int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 159.9+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4018 entries, 0 to 4017
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             4018 non-null   object 
 1   temp_max         4018 non-null   float64
 2   temp_min         4018 non-null   float64
 3   rain(mm)         4018 non-null   float64
 4   humidity_max(%)  4018 non-null   float64
 5   humidity_min(%)  4018 non-null   float64
 6   sunshine(MJ/m2)  4018 non-null   float64
dtypes: float64(6), object(1)
memory usage: 219.9+ KB


None

In [5]:
display(taxi.head())
display(weather.head())

,기준일,차량운행,접수건,탑승건,평균대기시간,평균요금,평균승차거리
0,2015-01-01,213,1023,924,23.2,2427,10764
1,2015-01-02,420,3158,2839,17.2,2216,8611
2,2015-01-03,209,1648,1514,26.2,2377,10198
3,2015-01-04,196,1646,1526,24.5,2431,10955
4,2015-01-05,421,4250,3730,26.2,2214,8663


,Date,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2)
0,2012-01-01,0.4,-6.6,0.0,77.0,45.0,4.90
1,2012-01-02,-1.2,-8.3,0.0,80.0,48.0,6.16
2,2012-01-03,-0.4,-6.6,0.4,86.0,45.0,4.46
3,2012-01-04,-4.6,-9.5,0.0,66.0,38.0,8.05
4,2012-01-05,-1.4,-9.6,0.0,71.0,28.0,9.14


#### 3) 칼럼이름을 영어로 변경
* 꼭 필요한 작업은 아니지만, 데이터를 편리하게 다루고 차트에서 불필요한 경고메시지를 띄우지 않게 하기 위해 영문으로 변경하기를 권장합니다.


In [90]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
taxi = taxi.rename(columns={
    '기준일': 'Date',
    '차량운행': 'car_operation',
    '접수건': 'booking_count',
    '탑승건': 'boarding_count',
    '평균대기시간': 'avg_wait_time',
    '평균요금': 'avg_fare',
    '평균승차거리': 'avg_distance'
})

taxi.head()

,Date,car_operation,booking_count,boarding_count,avg_wait_time,avg_fare,avg_distance
0,2015-01-01,213,1023,924,23.2,2427,10764
1,2015-01-02,420,3158,2839,17.2,2216,8611
2,2015-01-03,209,1648,1514,26.2,2377,10198
3,2015-01-04,196,1646,1526,24.5,2431,10955
4,2015-01-05,421,4250,3730,26.2,2214,8663


## 2.데이터 기본 탐색

* **세부요구사항**
    * 날짜 요소에 따라 각 정보의 패턴을 조회 합니다.
        * 일별, 요일별, 주차별, 월별, 연도별
        * 접수건, 탑승건, 거리, 요금, 대기시간 등
    * 제시된 범위 외에 가능하다면 추가 탐색을 시도합니다.

### (1) 주기별 분석을 위해서 날짜 변수 추가하기
* data를 복사합니다.
* 복사한 df에 요일, 주차, 월, 연도 등을 추가합니다.

In [91]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
data1 = taxi.copy()
data2 = weather.copy()

In [48]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
dt1 = pd.to_datetime(data1['Date'])
dt2 = pd.to_datetime(data2['Date'])

data1['weekday'] = dt1.dt.weekday
data1['month'] = dt1.dt.month
data1['day'] = dt1.dt.day
data1['week'] = dt1.dt.week
data1['year'] = dt1.dt.year

data2['weekday'] = dt2.dt.weekday
data2['month'] = dt2.dt.month
data2['day'] = dt2.dt.day
data2['week'] = dt2.dt.week
data2['year'] = dt2.dt.year

data1.head()

,Date,car_operation,booking_count,boarding_count,avg_wait_time,avg_fare,avg_distance,weekday,month,day,week,year
0,2015-01-01,213,1023,924,23.2,2427,10764,3,1,1,1,2015
1,2015-01-02,420,3158,2839,17.2,2216,8611,4,1,2,1,2015
2,2015-01-03,209,1648,1514,26.2,2377,10198,5,1,3,1,2015
3,2015-01-04,196,1646,1526,24.5,2431,10955,6,1,4,1,2015
4,2015-01-05,421,4250,3730,26.2,2214,8663,0,1,5,2,2015


In [23]:
data1.columns

Index(['Date', 'car_operation', 'booking_count', 'boarding_count',
       'avg_wait_time', 'avg_fare', 'avg_distance', 'weekday', 'month', 'day',
       'week', 'year'],
      dtype='object')

### (2) 일별

In [25]:
data1.groupby(by = 'day', as_index = False)[['car_operation', 'booking_count', 'boarding_count',
       'avg_wait_time', 'avg_fare', 'avg_distance']].agg(['mean'])

,car_operation,booking_count,boarding_count,avg_wait_time,avg_fare,avg_distance
,mean,mean,mean,mean,mean,mean
day,,,,,,
1,388.375000,3462.656250,2875.177083,37.095833,2325.781250,9481.916667
2,419.520833,3925.177083,3277.520833,37.564583,2290.927083,9128.427083
3,408.614583,3725.281250,3129.187500,36.615625,2312.458333,9331.250000
4,418.666667,3886.364583,3269.989583,38.382292,2302.812500,9242.875000
5,408.968750,3755.000000,3153.093750,38.445833,2309.729167,9310.510417
6,409.031250,3740.760417,3148.364583,36.551042,2307.687500,9294.281250
7,427.583333,4030.062500,3377.666667,40.040625,2294.854167,9169.781250
8,428.739583,4058.052083,3399.833333,41.313542,2299.104167,9199.520833


* 차량 운행수
    - 고르게 분포, 평균 420대 정도 운행

* 접수건, 탑승건
    - 접수에비해서 탑승수가 적은 편이다

* 대기시간
    - 평군 40분 정도 대기

* 운임, 이동거리
    - 평균 2300원, 9km정도 이동

### (3) 요일별

In [24]:
data1.groupby(by = 'weekday', as_index = False)[['car_operation', 'booking_count', 'boarding_count',
       'avg_wait_time', 'avg_fare', 'avg_distance']].agg(['mean'])

,car_operation,booking_count,boarding_count,avg_wait_time,avg_fare,avg_distance
,mean,mean,mean,mean,mean,mean
weekday,,,,,,
0,485.014388,4715.040767,3961.242206,39.243165,2240.059952,8656.577938
1,495.076739,4800.884892,4026.791367,40.826139,2242.805755,8672.112710
2,484.726619,4739.333333,3956.976019,41.676259,2250.422062,8752.038369
3,494.078947,4760.677033,3978.200957,41.043780,2256.366029,8786.901914
4,485.107656,4692.066986,3877.385167,40.956459,2246.023923,8733.665072
5,262.177033,2069.081340,1693.839713,43.480144,2429.641148,10431.468900
6,224.767386,1702.781775,1493.556355,34.902878,2464.541966,10741.275779


* 차량 운행수
    - 평일에 운행수가 주말에 비해 2배정도 많음

* 접수건, 탑승건
    - 접수건수도 주말에 비해 평일이 2배정도 많음
    - 탑승건수도 마찬가지

* 대기시간
    - 대기시간은 일요일을 제외하고는 평균 40분이상 소요

* 운임, 이동거리
    - 평균 2300원, 8 ~ 10km정도 이동(주말에 이동 거리 크다)

### (4) 월별

In [26]:
data1.groupby(by = 'month', as_index = False)[['car_operation', 'booking_count', 'boarding_count',
       'avg_wait_time', 'avg_fare', 'avg_distance']].agg(['mean'])

,car_operation,booking_count,boarding_count,avg_wait_time,avg_fare,avg_distance
,mean,mean,mean,mean,mean,mean
month,,,,,,
1,379.286290,3681.826613,3092.262097,33.393952,2279.616935,9048.161290
2,373.486726,3582.097345,3032.075221,34.419469,2296.623894,9221.039823
3,398.758065,3691.330645,3168.822581,35.182258,2291.600806,9129.588710
4,416.187500,3964.479167,3358.558333,39.628750,2310.712500,9313.458333
5,413.556452,3910.572581,3278.435484,41.181452,2326.209677,9462.447581
6,430.925000,4001.304167,3372.837500,40.381667,2308.991667,9292.650000
7,436.935484,4099.637097,3437.862903,40.033468,2290.407258,9151.556452
8,434.500000,4014.963710,3350.512097,38.568952,2286.189516,9127.072581


* 차량 운행수
    - 1 ~ 3월에는 상대적으로 적고 6 ~ 11월 운행수가 많다

* 접수건, 탑승건
    - 접수건수도 운행수와 비슷하다
    - 탑승건수도 마찬가지

* 대기시간
    - 대기시간은 1~ 3월에는 상대적으로 적지만, 6 ~ 11월은 평균적으로 40 ~ 50분 대기

* 운임, 이동거리
    - 운임: 평균 2300원
    - 이동거리: 평균 9km정도 이동

### (5) 연도별

In [27]:
data1.groupby(by = 'year', as_index = False)[['car_operation', 'booking_count', 'boarding_count',
       'avg_wait_time', 'avg_fare', 'avg_distance']].agg(['mean'])

,car_operation,booking_count,boarding_count,avg_wait_time,avg_fare,avg_distance
,mean,mean,mean,mean,mean,mean
year,,,,,,
2015,381.468493,3750.531507,3268.673973,34.059452,2301.523288,9558.975342
2016,390.314208,4005.087432,3418.183060,38.272404,2316.404372,9581.713115
2017,405.397260,4184.745205,3471.709589,44.143014,2321.558904,9311.230137
2018,397.758904,4077.473973,3209.380822,56.322192,2319.293151,9296.868493
2019,399.068493,4104.358904,3230.279452,53.455616,2316.846575,9258.534247
2020,416.521858,3137.480874,2656.314208,28.045082,2308.505464,9181.213115
2021,468.271233,3796.821918,3344.109589,30.720822,2270.000000,8864.536986
2022,490.898630,4350.304110,3673.863014,37.466027,2280.002740,8974.227397


* 차량 운행수
    - 연도가 지남에 따라 운행수가 점점 많아짐

* 접수건, 탑승건
    - 접수건수도 증가하다가 2020년에 급격히 감소후 다시 증가 (코로나에 의한 영향 같다)
    - 탑승건수도 마찬가지

* 대기시간
    - 대기시간은 점점 늘다가 2020년 급격히 감소후 다시 증가하는 중이다

* 운임, 이동거리
    - 운임: 평균 2300원
    - 이동거리: 평균 9km정도 이동

## 3.데이터 구조 만들기

* **세부요구사항**
    * 조건 : 
        * 목표 : 전날 저녁, 다음날 평균 대기시간을 예측하고자 합니다.
        * 날씨 데이터는 실제 측정값이지만, 다음 날에 대한 예보 데이터로 간주합니다. 
            * 예를 들어, 
                * 2020-12-23 의 날씨 데이터는 전날(12월22일) 날씨예보 데이터로 간주하여 분석을 수행합니다.
                * 2020-12-22일의 장애인 이동 데이터로 23일의 대기시간을 예측해야 하며, 이때 고려할 날씨데이터는 23일 데이터 입니다.
    * 장애인 이동 데이터를 기준으로 날씨 데이터를 붙입니다.

In [92]:
data1 = taxi.copy()
data2 = weather.copy()

In [93]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2922 entries, 0 to 2921
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            2922 non-null   object 
 1   car_operation   2922 non-null   int64  
 2   booking_count   2922 non-null   int64  
 3   boarding_count  2922 non-null   int64  
 4   avg_wait_time   2922 non-null   float64
 5   avg_fare        2922 non-null   int64  
 6   avg_distance    2922 non-null   int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 159.9+ KB


In [96]:
data2.shape

(4018, 7)

In [97]:
data2.head()

,Date,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2)
0,2012-01-01,0.4,-6.6,0.0,77.0,45.0,4.90
1,2012-01-02,-1.2,-8.3,0.0,80.0,48.0,6.16
2,2012-01-03,-0.4,-6.6,0.4,86.0,45.0,4.46
3,2012-01-04,-4.6,-9.5,0.0,66.0,38.0,8.05
4,2012-01-05,-1.4,-9.6,0.0,71.0,28.0,9.14


### (1) target 만들기
* 예측하는 날짜, 대기시간(target)으로 기준을 잡습니다.

In [94]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
# 익일의 대기시간(waiting time)을 오늘의 데이터를 활용하여 예측 해야하는 대상(target)으로 설정
data1['Date'] = pd.to_datetime(data1['Date'])
data2['Date'] = pd.to_datetime(data2['Date'])

In [95]:
data1['target'] = data1['avg_wait_time'].shift(-1)
data1.tail()

,Date,car_operation,booking_count,boarding_count,avg_wait_time,avg_fare,avg_distance,target
2917,2022-12-27,669,5635,4654,44.4,2198,8178,44.8
2918,2022-12-28,607,5654,4648,44.8,2161,7882,52.5
2919,2022-12-29,581,5250,4247,52.5,2229,8433,38.3
2920,2022-12-30,600,5293,4200,38.3,2183,8155,33.7
2921,2022-12-31,263,2167,1806,33.7,2318,9435,NaN


In [98]:
data2['Date'] = pd.to_datetime(data2['Date']) - pd.Timedelta(days=1)
display(data2.head())
display(data2.tail())

,Date,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2)
0,2011-12-31,0.4,-6.6,0.0,77.0,45.0,4.90
1,2012-01-01,-1.2,-8.3,0.0,80.0,48.0,6.16
2,2012-01-02,-0.4,-6.6,0.4,86.0,45.0,4.46
3,2012-01-03,-4.6,-9.5,0.0,66.0,38.0,8.05
4,2012-01-04,-1.4,-9.6,0.0,71.0,28.0,9.14


,Date,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2)
4013,2022-12-26,3.0,-7.3,0.0,86.0,51.0,10.25
4014,2022-12-27,-0.3,-5.4,0.1,92.0,40.0,10.86
4015,2022-12-28,1.7,-7.8,0.0,71.0,34.0,10.88
4016,2022-12-29,2.1,-4.0,0.0,87.0,38.0,10.84
4017,2022-12-30,-4.4,-4.4,0.0,66.0,66.0,0.00


### (2) 날씨 데이터 붙이기
* merge를 활용합니다. 기준은 운행정보 입니다.

In [116]:
data = pd.merge(data1, data2, on = 'Date')
data.tail()

,Date,car_operation,booking_count,boarding_count,avg_wait_time,avg_fare,avg_distance,target,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2)
2916,2022-12-26,603,5555,4605,39.2,2163,7889,44.4,3.0,-7.3,0.0,86.0,51.0,10.25
2917,2022-12-27,669,5635,4654,44.4,2198,8178,44.8,-0.3,-5.4,0.1,92.0,40.0,10.86
2918,2022-12-28,607,5654,4648,44.8,2161,7882,52.5,1.7,-7.8,0.0,71.0,34.0,10.88
2919,2022-12-29,581,5250,4247,52.5,2229,8433,38.3,2.1,-4.0,0.0,87.0,38.0,10.84
2920,2022-12-30,600,5293,4200,38.3,2183,8155,33.7,-4.4,-4.4,0.0,66.0,66.0,0.00


In [117]:
data.isna().sum()

Date               0
car_operation      0
booking_count      0
boarding_count     0
avg_wait_time      0
avg_fare           0
avg_distance       0
target             0
temp_max           0
temp_min           0
rain(mm)           0
humidity_max(%)    0
humidity_min(%)    0
sunshine(MJ/m2)    0
dtype: int64

### (3) 새로운 feature를 생성해 봅시다.
* 날짜와 관련된 변수 추가하기 : 요일, 월, 계절, 연도
* 그외 새로운 feature 도출 : 최소 2개 이상
    * 예 : 공휴일, 최근 7주일간의 평균 대기시간, 탑승률 등

#### 1) 날짜와 관련된 변수 추가하기 : 요일, 월, 계절, 연도
* 요일 이름, 계절이름, 월 이름으로 만드는 경우에는, 변수를 pd.Categorical로 범주형을 만들면서 순서를 지정하는 것이 이후 그래프를 그릴 때 순서대로 표현할 수 있습니다.


In [118]:
data['weekday'] = data['Date'].dt.day_name()
data['weekday'] = pd.Categorical(data['weekday'], 
                                  categories=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
data['month'] = data['Date'].dt.month

data['season'] = np.where(data['month'].isin([3,4,5]), 'Spring',
                           np.where(data['month'].isin([6,7,8]), 'Summer',
                                    np.where(data['month'].isin([9,10,11]), 'Fall', 'Winter')))
data['season'] = pd.Categorical(data['season'], categories=['Spring','Summer','Fall','Winter'])
                                     
data['year'] = data['Date'].dt.year
data.head()

,Date,car_operation,booking_count,boarding_count,avg_wait_time,avg_fare,avg_distance,target,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2),weekday,month,season,year
0,2015-01-01,213,1023,924,23.2,2427,10764,17.2,-2.0,-8.9,0.0,63.0,28.0,9.07,Thursday,1,Winter,2015
1,2015-01-02,420,3158,2839,17.2,2216,8611,26.2,2.4,-9.2,0.0,73.0,37.0,8.66,Friday,1,Winter,2015
2,2015-01-03,209,1648,1514,26.2,2377,10198,24.5,8.2,0.2,0.0,89.0,58.0,5.32,Saturday,1,Winter,2015
3,2015-01-04,196,1646,1526,24.5,2431,10955,26.2,7.9,-0.9,0.0,95.0,52.0,6.48,Sunday,1,Winter,2015
4,2015-01-05,421,4250,3730,26.2,2214,8663,23.6,4.1,-7.4,3.4,98.0,29.0,10.47,Monday,1,Winter,2015


#### 2) 공휴일 정보
* workalendar 패키지를 설치하고, 대한민국 공휴일 정보를 끌어와 봅시다.

* 휴무일 데이터 패키지 설치

In [74]:
%pip install workalendar

     -------------------------------------- 210.7/210.7 kB 6.3 MB/s eta 0:00:00
     ---------------------------------------- 47.9/47.9 kB 2.5 MB/s eta 0:00:00
     ------------------------------------- 345.4/345.4 kB 10.5 MB/s eta 0:00:00
     ---------------------------------------- 5.8/5.8 MB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pymeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=732005 sha256=df033144d66c39b03f732ec13d7b5dbd2f72bb27f190ddc03940d0cc76d79602
  Stored in directory: c:\users\pjj11\appdata\local\pip\cache\wheels\80\11\82\146513e81cf4efcc994780a5f4c25387d66894c4d139c62b74
Successfully built pymeeus
Note: you may need to restart the kernel to use updated packages.


* 간단 사용법

In [119]:
from workalendar.asia import SouthKorea
cal = SouthKorea()
pd.DataFrame(cal.holidays(2023))

,0,1
0,2023-01-01,New year
1,2023-01-21,Korean New Year's Day
2,2023-01-22,Korean New Year's Day
3,2023-01-23,Korean New Year's Day
4,2023-03-01,Independence Day
5,2023-05-05,Children's Day
6,2023-05-26,Buddha's Birthday
7,2023-06-06,Memorial Day
8,2023-08-15,Liberation Day
9,2023-09-28,Midautumn Festival


* 휴무일 데이터셋 만들기 2015 ~ 2022
* 실제로 휴무일에 해당하지만 workalendar 라이브러리에 없는 날짜는 직접 추가해봅시다.
    * 휴무일 장애인 콜택시의 접수건 변화에 대한 특징을 찾아 이를 바탕으로 데이터를 조회하여 찾아볼 수 있음

In [120]:
from workalendar.asia import SouthKorea

cal = SouthKorea()
holiday = pd.DataFrame()
for y in range(2015, 2023) :
    holiday = pd.concat([holiday, pd.DataFrame(cal.holidays(y))], axis = 0)

holiday.columns = ['Date', 'holiday']
holiday['Date'] = pd.to_datetime(holiday['Date'])
holiday['holiday'] = 1
holiday.head()

,Date,holiday
0,2015-01-01,1
1,2015-02-18,1
2,2015-02-19,1
3,2015-02-20,1
4,2015-03-01,1


* 기존 데이터에 휴무일 정보 결합하기.
* 휴무일이 아닌 날짜는 0으로 저장하시오.

In [121]:
data = pd.merge(data, holiday, how = 'left')
data = data.fillna({'holiday':0})
data.head()

,Date,car_operation,booking_count,boarding_count,avg_wait_time,avg_fare,avg_distance,target,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2),weekday,month,season,year,holiday
0,2015-01-01,213,1023,924,23.2,2427,10764,17.2,-2.0,-8.9,0.0,63.0,28.0,9.07,Thursday,1,Winter,2015,1.0
1,2015-01-02,420,3158,2839,17.2,2216,8611,26.2,2.4,-9.2,0.0,73.0,37.0,8.66,Friday,1,Winter,2015,0.0
2,2015-01-03,209,1648,1514,26.2,2377,10198,24.5,8.2,0.2,0.0,89.0,58.0,5.32,Saturday,1,Winter,2015,0.0
3,2015-01-04,196,1646,1526,24.5,2431,10955,26.2,7.9,-0.9,0.0,95.0,52.0,6.48,Sunday,1,Winter,2015,0.0
4,2015-01-05,421,4250,3730,26.2,2214,8663,23.6,4.1,-7.4,3.4,98.0,29.0,10.47,Monday,1,Winter,2015,0.0


#### 3) 7일 이동평균 대기시간
* rolling().mean() 사용

In [122]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
data['avg_wait_time_7'] = data['avg_wait_time'].rolling(7).mean()
data.fillna(method = 'bfill', inplace =True)
data.head(8)

,Date,car_operation,booking_count,boarding_count,avg_wait_time,avg_fare,avg_distance,target,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2),weekday,month,season,year,holiday,avg_wait_time_7
0,2015-01-01,213,1023,924,23.2,2427,10764,17.2,-2.0,-8.9,0.0,63.0,28.0,9.07,Thursday,1,Winter,2015,1.0,23.657143
1,2015-01-02,420,3158,2839,17.2,2216,8611,26.2,2.4,-9.2,0.0,73.0,37.0,8.66,Friday,1,Winter,2015,0.0,23.657143
2,2015-01-03,209,1648,1514,26.2,2377,10198,24.5,8.2,0.2,0.0,89.0,58.0,5.32,Saturday,1,Winter,2015,0.0,23.657143
3,2015-01-04,196,1646,1526,24.5,2431,10955,26.2,7.9,-0.9,0.0,95.0,52.0,6.48,Sunday,1,Winter,2015,0.0,23.657143
4,2015-01-05,421,4250,3730,26.2,2214,8663,23.6,4.1,-7.4,3.4,98.0,29.0,10.47,Monday,1,Winter,2015,0.0,23.657143
5,2015-01-06,417,3991,3633,23.6,2211,8545,24.7,-1.0,-8.8,0.0,42.0,24.0,10.12,Tuesday,1,Winter,2015,0.0,23.657143
6,2015-01-07,410,4085,3676,24.7,2230,8646,21.2,-0.2,-9.2,0.0,62.0,27.0,10.09,Wednesday,1,Winter,2015,0.0,23.657143
7,2015-01-08,419,4030,3728,21.2,2231,8683,21.8,3.2,-6.8,0.0,78.0,38.0,8.74,Thursday,1,Winter,2015,0.0,23.371429


#### 4) 탑승률

In [123]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
data['boarding_rate'] = data['boarding_count'] / data['booking_count'] # 탑승건수 / 접수 건수
data.head()

,Date,car_operation,booking_count,boarding_count,avg_wait_time,avg_fare,avg_distance,target,temp_max,temp_min,...,humidity_max(%),humidity_min(%),sunshine(MJ/m2),weekday,month,season,year,holiday,avg_wait_time_7,boarding_rate
0,2015-01-01,213,1023,924,23.2,2427,10764,17.2,-2.0,-8.9,...,63.0,28.0,9.07,Thursday,1,Winter,2015,1.0,23.657143,0.903226
1,2015-01-02,420,3158,2839,17.2,2216,8611,26.2,2.4,-9.2,...,73.0,37.0,8.66,Friday,1,Winter,2015,0.0,23.657143,0.898987
2,2015-01-03,209,1648,1514,26.2,2377,10198,24.5,8.2,0.2,...,89.0,58.0,5.32,Saturday,1,Winter,2015,0.0,23.657143,0.918689
3,2015-01-04,196,1646,1526,24.5,2431,10955,26.2,7.9,-0.9,...,95.0,52.0,6.48,Sunday,1,Winter,2015,0.0,23.657143,0.927096
4,2015-01-05,421,4250,3730,26.2,2214,8663,23.6,4.1,-7.4,...,98.0,29.0,10.47,Monday,1,Winter,2015,0.0,23.657143,0.877647


In [124]:
data.loc[data['Date'] == '2017-10-03']

,Date,car_operation,booking_count,boarding_count,avg_wait_time,avg_fare,avg_distance,target,temp_max,temp_min,...,humidity_max(%),humidity_min(%),sunshine(MJ/m2),weekday,month,season,year,holiday,avg_wait_time_7,boarding_rate
1006,2017-10-03,260,1676,1352,24.2,2567,11747,24.6,24.2,10.7,...,65.0,33.0,9.93,Tuesday,10,Fall,2017,1.0,53.571429,0.806683
1007,2017-10-03,260,1676,1352,24.2,2567,11747,24.6,24.2,10.7,...,65.0,33.0,9.93,Tuesday,10,Fall,2017,1.0,47.928571,0.806683


In [125]:
data.drop(1007, inplace=True)
data['Date'].value_counts()

2015-01-01    1
2020-04-24    1
2020-04-26    1
2020-04-27    1
2020-04-28    1
             ..
2017-09-03    1
2017-09-04    1
2017-09-05    1
2017-09-06    1
2022-12-30    1
Name: Date, Length: 2921, dtype: int64

In [126]:
data.shape

(2921, 21)

## 4.데이터 저장
* **세부요구사항**
    * joblib 을 사용하여 작업 경로에 정리한 데이터프레임을 저장합니다.
        * 저장파일이름 : data1.pkl

In [129]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
data.to_csv('data1.csv', index=False)